# Optimize Model.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

In [ ]:
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))
X = X/255.0

In [ ]:
dense_layers = [0,1,2]
layer_sizes = [32,64,128] # convention. one on each size.
conv_layers = [1,2,3]

In [ ]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(
                conv_layer,layer_size,dense_layer,
                int(time.time()))
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model = Sequential()

            model.add(Conv2D(layer_size,(3,3),input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten())

            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))
                model.add(Dropout(0,2))

            # output layer.
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(optimizer="adam",
                        loss="binary_crossentropy",
                        metrics=['accuracy'])

            model.fit(X, y,
                    batch_size=32, epochs=5,
                    validation_split=0.1,
                    callbacks=[tensorboard])